In [1]:
import pandas as pd
%pylab inline
from datetime import datetime
from datetime import date
from datetime import timedelta

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
%pylab inline
from datetime import datetime
from datetime import timedelta

# Page Impressions
piDat = pd.read_csv('../data/PI_angereicherteDaten.csv.txt',sep='\t',low_memory=False,na_values=['-'])
# Visits
vDat = pd.read_csv('../data/V_angereicherteDaten.txt',sep='\t',low_memory=False,na_values=['-'])

# contract data from contracts which are of interest to us
contract = pd.read_pickle('contract_data') 

# date handling
dateparse1 = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
dateparse2 = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')

# from piDat and vDat make pivot tables for each KPI of interest such that each line represents one user, 
# columns represent weeks
#
# filter for subscriptions of interest ('Vollabo' & WeltPlus etc., no strange cancellation reasons)
#

# lists of variable names for new pivot tbales and lists of colnumns names in original data
list_pi_varNames = ['v','vPlus','pi','piComment','piSection','piArticle','piPlusComment','piPlusArticle']
list_pi_colNames = ['Visits', 'Visits Plus',
 'Page Impressions', 'Page Impressions plus' ,'PI [mit Komment]',
 'PI [Typ: Section]', 'PI [Typ: Article] ', 'PI Plus [mit Komment]',
 'PI Plus [Typ: Section]' ,'PI Plus [Typ: Article]']

list_v_varNames = ['vComment','vSection','vArticle','vMobile','vDesktop',
                   'vPlusComment','vPlusArticle','vPlusMobile','vPlusDesktop']
list_v_colNames = ['Visits [mit Komment]', 'Visits [mit Section] ','Visits [mit Article] ', 
                   'Visits [Mobile] ', 'Visits [Desktop] ', 
                   'Visits Plus [mit Komment]', 'Visits Plus [mit Article]',
                   'Visits Plus [Mobile]', 'Visits Plus [Desktop]']

# list of IDs from subscriptions of interest
conIdx = contract['WT SSOID']


# contains in the end all pivot tables
pivot_tables_pi_v = {}

# from piDat
for idx, val in enumerate(list_pi_varNames):
    # make pivot table
    table = pd.pivot_table(piDat,values=list_pi_colNames[idx], 
                                  index='URM - Eigene Besucher-ID', columns='Wochen',aggfunc='first')
    # filter
    pivot_tables_pi_v[val] = table.loc[table.index.isin(conIdx)]
    
# fom vDat
for idx, val in enumerate(list_v_varNames):
    # make pivot table
    table = pd.pivot_table(vDat, values=list_v_colNames[idx], 
                                  index='URM - Eigene Besucher-ID', columns='Wochen',aggfunc='first')
    # filter
    pivot_tables_pi_v[val] = table.loc[table.index.isin(conIdx)]

    
# make relations with v
list_total = list_pi_varNames[1:]+list_v_varNames
for idx, val in enumerate(list_total):
    # name to save table
    table_name= val + '_per_v'
    # make relations
    pivot_tables_pi_v[table_name] = pivot_tables_pi_v[val]/pivot_tables_pi_v['v']



Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [85]:
vtab=pivot_tables_pi_v['pi_per_v']

2016

In [101]:
def test_if_classification_works(dat, begin_week, begin_year, end_week, end_year):
    
    #print(dat.name)
    con_begin = contract.loc[dat.name,'VB Tag']
    con_canc = contract.loc[dat.name,'K Tag']
    
    one_month_after_canc_day = con_begin+timedelta(days=30) # one_month_after_canc_day = 'OMacD'
    oMacD_week = one_month_after_canc_day.weekofyear
    oMacD_year = one_month_after_canc_day.year
    
    if (oMacD_year<begin_year) | ((oMacD_year==begin_year) & (oMacD_week < (begin_week + 4)) ):
        oMacD_week = begin_week + 4
        oMacD_year = begin_year
    
    # make list of weeks from
    all_weeks=list(dat.index)
    all_weeks=[x.split('_')[0] for x in all_weeks if len(x.split('_'))==1 ]
    
    first_idx = str(oMacD_year) + '/' + str((oMacD_week+1)).zfill(2)
    if first_idx=='2017/53':
        first_idx = '2018/01'
    first = all_weeks.index(first_idx)
    
    
    weeks_to_iterate = all_weeks[first:]
    
    for week in weeks_to_iterate:
        
        week_now = int(week.split('/')[1])
        year_now = int(week.split('/')[0])
        
        # preparation
        idxWeek = all_weeks.index(week)    
        week_before = all_weeks[idxWeek-1]
        
        mean_until_now = dat.loc[all_weeks[:idxWeek]].mean()
        median_until_now = dat.loc[all_weeks[:idxWeek]].median()
        sd_until_now = dat.loc[all_weeks[:idxWeek]].std()
        
        #print(week)
        
        zsc = (dat.loc[week] - mean_until_now )/sd_until_now 
        prz = dat.loc[week]/median_until_now 
        
        week_before_zsc_flag = all_weeks[idxWeek-1] + '_zsc_flag'
        week_now_zsc_flag = week + '_zsc_flag'
        week_before_prz_flag = all_weeks[idxWeek-1] + '_prz_flag'
        week_now_prz_flag = week + '_prz_flag'
        
        week_now_zsc_churn = week + '_zsc_churn'
        week_now_prz_churn = week + '_prz_churn'
        
              
        # setting flags for alert: prz
        if (zsc < 2) & (dat[week_before_zsc_flag] == 'empty'):
            dat[week_now_zsc_flag] = 'flag1'
        if (zsc < 2) & (dat[week_before_zsc_flag] == 'flag1'):
            dat[week_now_zsc_flag] = 'flag2'
        if (zsc < 2) & (dat[week_before_zsc_flag] == 'flag2'):
            dat[week_now_zsc_flag] = 'flag2'
            
            # flag indicates churn            
            
            # comparison if flag correctly set -> different cases
            
            # 1st there is no cancellation at all: 'wrong'
            if pd.isnull(contract.loc[dat.name,'K Tag']):
                dat[week_now_zsc_churn] = 'wrong'
            
            else:
                # 2nd look at cancellation date
                week_canc = con_canc.weekofyear 
                year_canc = con_canc.year
            
            
                if ((year_now - year_canc)==0) & ((week_canc-week_now)<-1):
                    dat.loc[week_now_zsc_churn] = 'too_late'
                elif ((year_now - year_canc)==0) & ((week_canc-week_now)>4):
                    dat.loc[week_now_zsc_churn] = 'too_early'
                else:
                    dat.loc[week_now_zsc_churn] = 'correct'
            
        
       
        # setting flags for alert: zsc
        if (prz < 2) & (dat[week_before_prz_flag] == 'empty'):
            dat.loc[week_now_prz_flag] = 'flag1'
        if (prz < 2) & (dat[week_before_prz_flag] == 'flag1'):
            dat.loc[week_now_prz_flag] = 'flag2'
        if (prz < 2) & (dat[week_before_prz_flag] == 'flag2'):
            dat.loc[week_now_prz_flag] = 'flag2'
            
            # flag indicates churn            
            
            # comparison if flag correctly set -> different cases
            
            # 1st there is no cancellation at all: 'wrong'
            if pd.isnull(contract.loc[dat.name,'K Tag']):
                dat.loc[week_now_prz_churn] = 'wrong'
            
            else:            
                # 2nd look at cancellation date
                week_canc = con_canc.weekofyear 
                year_canc = con_canc.year
                        
                if (((year_now - year_canc)==0) & ((week_canc-week_now)<-1)):
                    dat.loc[week_now_prz_churn] = 'too_late'
                elif (((year_now - year_canc)==0) & ((week_canc-week_now)>4)):
                    dat[week_now_prz_churn] = 'too_early'
                else:
                    dat.loc[week_now_prz_churn] = 'correct'
                    
                          
    return dat

In [40]:
cols = vtab.columns.values
cols_prz_flag = [x+'_prz_flag' for x in cols]
cols_zsc_flag = [x+'_zsc_flag' for x in cols]

cols_prz_churn = [x+'_prz_churn' for x in cols]
cols_zsc_churn = [x+'_zsc_churn' for x in cols]

col_tot = cols_prz_flag +cols_zsc_flag+cols_prz_churn+cols_zsc_churn

vdat = vtab.copy()

for col in col_tot:
    vdat[col] = 'empty'
    


In [102]:
print(datetime.now())
vdat = vdat.apply(test_if_classification_works, axis=1, args=(47, 2017, 21, 2018))
print(datetime.now())

2018-06-08 17:03:45.875345


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1018: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


2018-06-08 17:10:35.867241


In [103]:
vdat.columns.values

array(['2017/47', '2017/48', '2017/49', '2017/50', '2017/51', '2017/52',
       '2018/01', '2018/02', '2018/03', '2018/04', '2018/05', '2018/06',
       '2018/07', '2018/08', '2018/09', '2018/10', '2018/11', '2018/12',
       '2018/13', '2018/14', '2018/15', '2018/16', '2018/17', '2018/18',
       '2018/19', '2018/20', '2018/21', '2017/47_prz_flag',
       '2017/48_prz_flag', '2017/49_prz_flag', '2017/50_prz_flag',
       '2017/51_prz_flag', '2017/52_prz_flag', '2018/01_prz_flag',
       '2018/02_prz_flag', '2018/03_prz_flag', '2018/04_prz_flag',
       '2018/05_prz_flag', '2018/06_prz_flag', '2018/07_prz_flag',
       '2018/08_prz_flag', '2018/09_prz_flag', '2018/10_prz_flag',
       '2018/11_prz_flag', '2018/12_prz_flag', '2018/13_prz_flag',
       '2018/14_prz_flag', '2018/15_prz_flag', '2018/16_prz_flag',
       '2018/17_prz_flag', '2018/18_prz_flag', '2018/19_prz_flag',
       '2018/20_prz_flag', '2018/21_prz_flag', '2017/47_zsc_flag',
       '2017/48_zsc_flag', '2017/49_zsc_flag'

In [124]:
sum(vdat.iloc[:,82 : 107]=='wrong')
#list(vdat.columns.values).index('2018/21_prz_churn')

Wochen
2017/48_prz_churn       0
2017/49_prz_churn       0
2017/50_prz_churn       0
2017/51_prz_churn       0
2017/52_prz_churn       0
2018/01_prz_churn       0
2018/02_prz_churn    2937
2018/03_prz_churn    3153
2018/04_prz_churn    3301
2018/05_prz_churn    3427
2018/06_prz_churn    3535
2018/07_prz_churn    3637
2018/08_prz_churn    3809
2018/09_prz_churn    3883
2018/10_prz_churn    3897
2018/11_prz_churn    3901
2018/12_prz_churn    3878
2018/13_prz_churn    3825
2018/14_prz_churn    3773
2018/15_prz_churn    3746
2018/16_prz_churn    3712
2018/17_prz_churn    3684
2018/18_prz_churn    3624
2018/19_prz_churn    3613
2018/20_prz_churn    3601
dtype: int64